Final Project Vignette Examples

In [ ]:
# libraries
import requests
import os
import pandas as pd
import json
import math # hmm
import numpy as np

In [ ]:
bc_api_key = os.getenv("POETRY_BC_LIBRARY_API_KEY")
print(bc_api_key[:5])

In [ ]:
def collections(*searches, api_key="ac40e6c2cb345593ed1691e0a8b601bba398e42d85f81f893c5ab709cec63c6c"):
    # 1. api_key
    if api_key=="ac40e6c2cb345593ed1691e0a8b601bba398e42d85f81f893c5ab709cec63c6c":
        print('This function utilizes the British Columbia Library public API Key by default, '
              'which limits requests to 10 per minute. '
              'Please register for a free private API Key to make up to 200 requests per minute. \n')
    
    # 2. 
    params = {'api_key':api_key}
    r = requests.get(f'https://oc2-index.library.ubc.ca/collections', params=params)
    print('The status of this request is: ', r.status_code, '. Please wait. \n')
    rseries = pd.DataFrame(r.json()).loc[:, ['data']].applymap(str) # change request into dataframe and make it str, to enable merge
    
    # 3. get metadata
    items_dict = {}
    i = 0
    for name in rseries['data'][:10]:
        ritems = requests.get(f'https://oc2-index.library.ubc.ca/collections/{str(name)}', params=params).json()['data']
        items_dict[i] = [name, ritems['title'], ritems['description'], ritems['items']]
        i += 1
     
    # 4. check
    for search in searches:
        if search in ritems['title'] or search in rseries['data'].any():
            print(f'{search} is a currently listed collection. \n')
        else:
            print(f'{search} is not a current collection in the University of British Columbia Library. \n'
                  'Please check your spelling or check the output to see available collections.')    
        
    items_df = pd.DataFrame.from_dict(items_dict, orient='index',
                                      columns=['CollectionID','CollectionName', 'description', 'items'])
    items_clean = items_df.replace(to_replace=[r'</?(p|span|title|i|a)(\s(class|style)="\w*\W*?.?")?>|\r\n|&#\d{1,5}'], value=[''], regex=True)
    
    return items_clean
x = collections("aaah", "xxx", api_key=bc_api_key)
x